## ДЗ 11: сравниваем ассоциаты одного и того же слова в разных корпусах

Задание: попробовать обучить две векторные модели на двух различных корпусах, которые можно сравнить между собой. 

Например: 

*   тексты газет из 1900 года (первая модель) и тексты газет из наших дней (вторая модель)
*   собрание сочинений одного писателй и собрание сочинений другого писателя (писатели должны быть сопоставимы: одно время работы, один тип текстов, один язык)

Найдите не меньше 3 слов, которые имеют ощутимо различные наборы семантических ассоциатов в двух моделях. Постарайтесь найти такие слова, различия между которыми могут быть проинтерпретированы и могут что-то говорить о разнице корпусов. 

**Важно: размер каждого корпуса должен быть не меньше 20 МБ.**

### 1. Загружаем корпус 1:

In [24]:
!wget https://github.com/hohlamon/model_news/raw/main/meduza_2.txt

--2022-05-03 10:32:17--  https://github.com/hohlamon/model_news/raw/main/meduza_2.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hohlamon/model_news/main/meduza_2.txt [following]
--2022-05-03 10:32:17--  https://raw.githubusercontent.com/hohlamon/model_news/main/meduza_2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76710904 (73M) [text/plain]
Saving to: ‘meduza_2.txt.2’

meduza_2.txt.2      100%[===================>]  73.16M   175MB/s    in 0.4s    

2022-05-03 10:32:18 (175 MB/s) - ‘meduza_2.txt.2’ saved [76710904/76710904]



In [ ]:
!wget https://github.com/hohlamon/model_news/raw/main/rt_2.txt

In [25]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install spacy==3.2

In [29]:
import spacy


In [ ]:
!python -m spacy download ru_core_news_md

In [31]:
russian_model = spacy.load('ru_core_news_md', disable=['ner', 'attribute_ruler'])

In [56]:
filename = 'rt_2.txt' # свой файл надо положить рядом и прописать тут путь
## выгрузим текст из файла
with open(filename, encoding='utf-8') as open_file:
    text_as_lines = open_file.read()

In [57]:
sent_text = sent_tokenize(text_as_lines)

### 2. Предобрабатываем 

(не обязательно использовать именно spacy)

Достаточно большой корпус для векторной модели может и обрабатываться достаточно долго. Например, на 14 русских романов у меня ушло минут 20 здесь.

Впрочем, вы можете подумать о том, как оптимизировать предобработку. Или не думать -- и просто дать компьютеру поработать часок-другой. 

In [32]:
def poser (sent):
  output = []
  parsed = russian_model(sent)
  for word in parsed:
    output.append(word.lemma_+'_'+word.pos_)
  return output

In [58]:
procesed_text = [poser(sent) for sent in sent_text]

In [ ]:
filename = 'meduza_2.txt' # свой файл надо положить рядом и прописать тут путь
## выгрузим текст из файла
with open(filename, encoding='utf-8') as open_file:
    text_as_lines = open_file.read()

In [ ]:
sent_text_2 = sent_tokenize(text_as_lines)

In [ ]:
procesed_text_2 = [poser(sent) for sent in sent_text]

Такое можно подавать в word2vec

### 3.Обучаем на этом модель

In [36]:
import gensim

In [65]:
modelRT = gensim.models.Word2Vec(procesed_text)

In [66]:

model.save("model_rt.wordvectors")


In [ ]:
modelMED = gensim.models.Word2Vec(procesed_text_2)

In [ ]:
model.save("model_med.wordvectors")

Теперь моделью можно пользоваться:

In [69]:
modelRT.wv.most_similar('оппозиционер_NOUN', topn = 20)

[('повстанец_NOUN', 0.7110620737075806),
 ('мусульманин_NOUN', 0.6926690936088562),
 ('мошенник_NOUN', 0.6732779145240784),
 ('либерал_NOUN', 0.6621700525283813),
 ('хуситы_NOUN', 0.6608474254608154),
 ('хуситов_NOUN', 0.660496175289154),
 ('радикал_NOUN', 0.6563068628311157),
 ('похититель_NOUN', 0.651795506477356),
 ('шиит_NOUN', 0.6398146748542786),
 ('оппозиция_NOUN', 0.6315505504608154),
 ('экстремист_NOUN', 0.6302471160888672),
 ('ополченец_NOUN', 0.6236810088157654),
 ('багдади_NOUN', 0.6232008934020996),
 ('одномандатник_NOUN', 0.621712863445282),
 ('социал_NOUN', 0.6202377676963806),
 ('аль_NOUN', 0.617321252822876),
 ('трансгендерами_NOUN', 0.6142008304595947),
 ('националист_NOUN', 0.6110759973526001),
 ('ккк_PROPN', 0.6084008812904358),
 ('хуситами_NOUN', 0.607941210269928)]

In [ ]:
modelMED.wv.most_similar('оппозиционер_NOUN', topn = 20)

### Теперь проделываем то же самое для другого корпуса — и сравниваем слова в двух моделях. Описываем результаты и интерпретируем их здесь же в тетрадке.